In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import logging
from datetime import datetime

<IPython.core.display.Javascript object>

In [3]:
import boto3
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker import image_uris
from sagemaker import hyperparameters

<IPython.core.display.Javascript object>

In [4]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

<IPython.core.display.Javascript object>

In [5]:
region = sagemaker.Session().boto_region_name
print(region)

ap-southeast-2


<IPython.core.display.Javascript object>

In [6]:
# role_arn = sagemaker.get_execution_role()
role_arn = os.getenv("SGMKR_ROLE_ARN")

<IPython.core.display.Javascript object>

In [7]:
bucket = "sgmkr-course"
prefix = "iris"

<IPython.core.display.Javascript object>

In [8]:
!aws s3 ls {bucket}/{prefix}/

                           PRE batch_transform/
                           PRE data/
                           PRE model/
2023-02-03 13:15:44          0 


<IPython.core.display.Javascript object>

In [9]:
!aws s3 ls {bucket}/{prefix}/data/ --recursive

2023-02-03 13:16:32          0 iris/data/
2023-03-16 18:09:28        900 iris/data/iris_test.csv
2023-03-16 18:09:27       1800 iris/data/iris_train.csv


<IPython.core.display.Javascript object>

In [10]:
train_file = "data/iris_train.csv"
valid_file = "data/iris_test.csv"

train_file_uri = "s3://{}/{}/{}".format(bucket, prefix, train_file)
valid_file_uri = "s3://{}/{}/{}".format(bucket, prefix, valid_file)
print("train file uri:", train_file_uri)
print("valid file uri:", valid_file_uri)

train file uri: s3://sgmkr-course/iris/data/iris_train.csv
valid file uri: s3://sgmkr-course/iris/data/iris_test.csv


<IPython.core.display.Javascript object>

In [11]:
train_ip = TrainingInput(train_file_uri, content_type="csv")
print(train_ip)

<IPython.core.display.Javascript object>

In [12]:
valid_ip = TrainingInput(valid_file_uri, content_type="csv")
print(valid_ip)

<IPython.core.display.Javascript object>

In [13]:
model_op = "s3://{}/{}/{}".format(bucket, prefix, "model")
print(model_op)

s3://sgmkr-course/iris/model


<IPython.core.display.Javascript object>

In [14]:
model_img = sagemaker.image_uris.retrieve("xgboost", region, "latest")
print(model_img)

Ignoring unnecessary instance type: None.


544295431143.dkr.ecr.ap-southeast-2.amazonaws.com/xgboost:latest


<IPython.core.display.Javascript object>

In [15]:
base_job_name = "iris-xgboost-"

<IPython.core.display.Javascript object>

In [16]:
xgb_model = sagemaker.estimator.Estimator(
    image_uri=model_img,
    role=role_arn,
    base_job_name=base_job_name,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=model_op,
    sagemaker_session=sagemaker.Session(),
    volume_size=5,
)

<IPython.core.display.Javascript object>

In [17]:
xgb_model.set_hyperparameters(
    num_class=3, max_depth=5, num_round=10, objective="multi:softmax",
)

<IPython.core.display.Javascript object>

In [18]:
job_name = base_job_name + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(job_name)

iris-xgboost2023-03-22-18-09-35


<IPython.core.display.Javascript object>

In [19]:
xgb_model.fit({"train": train_ip, "validation": valid_ip}, wait=True, job_name=job_name)

Creating training-job with name: iris-xgboost2023-03-22-18-09-35


2023-03-22 07:09:37 Starting - Starting the training job...
2023-03-22 07:10:02 Starting - Preparing the instances for training......
2023-03-22 07:11:02 Downloading - Downloading input data...
2023-03-22 07:11:32 Training - Downloading the training image...
2023-03-22 07:12:02 Training - Training image download completed. Training in progress..Arguments: train
[2023-03-22:07:12:17:INFO] Running standalone xgboost training.
[2023-03-22:07:12:17:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8604.3mb
[2023-03-22:07:12:17:INFO] Determined delimiter of CSV input is ','
[07:12:17] S3DistributionType set as FullyReplicated
[07:12:17] 100x4 matrix with 400 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-03-22:07:12:17:INFO] Determined delimiter of CSV input is ','
[07:12:17] S3DistributionType set as FullyReplicated
[07:12:17] 50x4 matrix with 200 entries loaded from /opt/ml/input/data/validation?format=csv

<IPython.core.display.Javascript object>

In [20]:
!aws s3 ls {bucket}/{prefix}/model/

                           PRE iris-xgboost-2023-02-03-06-03-15/
                           PRE iris-xgboost-2023-03-10-06-20-53/
                           PRE iris-xgboost-2023-03-10-17-15-32/
                           PRE iris-xgboost-2023-03-10-17-25-29/
                           PRE iris-xgboost2023-03-22-18-09-35/


<IPython.core.display.Javascript object>

In [21]:
!aws s3 ls {bucket}/{prefix}/model/{job_name}/

                           PRE debug-output/
                           PRE output/
                           PRE profiler-output/


<IPython.core.display.Javascript object>